## Preprocessing

In [125]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np
import os

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [126]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_2 = application_df.drop(['EIN', 'NAME'], axis=1).copy()

application_df_2.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [127]:
# Determine the number of unique values in each column.
application_df_2.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [128]:
# Look at APPLICATION_TYPE value counts for binning
application_df_2['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [129]:
# Choose a cutoff value and create a list of application types to be replaced

application_types_to_replace = []
app_counts = application_df_2['APPLICATION_TYPE'].value_counts()

for x in range(0,len(app_counts)):
    
    if app_counts[x] < 250:
        application_types_to_replace.append(app_counts.index[x])


# Replace in dataframe
for app in application_types_to_replace:
    application_df_2['APPLICATION_TYPE'] = application_df_2['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df_2['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [130]:
# Look at CLASSIFICATION value counts for binning
application_df_2['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1370        1
C1900        1
C2190        1
C3700        1
C6100        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [131]:
# CLASSIFICATION value counts >1
application_df_2['CLASSIFICATION'].value_counts().loc[lambda x: x > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1278       10
C1238       10
C1237        9
C1235        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C0           3
C2710        3
C1246        2
C1267        2
C1234        2
C1256        2
C3200        2
Name: CLASSIFICATION, dtype: int64

In [132]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classifications_to_replace = []
class_counts = application_df_2['CLASSIFICATION'].value_counts()

for x in range(0,len(class_counts)):
    
    if class_counts[x] < 1000:
        classifications_to_replace.append(class_counts.index[x])

# Replace in dataframe
for cls in classifications_to_replace:
    application_df_2['CLASSIFICATION'] = application_df_2['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df_2['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [133]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies_df = pd.get_dummies(application_df_2)

application_dummies_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [134]:
# Split our preprocessed data into our features and target arrays
X = application_dummies_df.drop(['IS_SUCCESSFUL'], axis=1)
y = application_dummies_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [135]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [136]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

input_dim = len(X_train_scaled[0])

# First hidden layer
first_units = 2*input_dim
nn.add(tf.keras.layers.Dense(units=first_units, activation = 'relu', input_dim=input_dim))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=first_units/2, activation = 'relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=int(first_units/4), activation = 'relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 86)                3784      
_________________________________________________________________
dense_13 (Dense)             (None, 43)                3741      
_________________________________________________________________
dense_14 (Dense)             (None, 21)                924       
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 22        
Total params: 8,471
Trainable params: 8,471
Non-trainable params: 0
_________________________________________________________________


In [137]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [144]:

# create checkpoint callbacks to save model's weights
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath='training_1/weights{epoch:04d}-acc{accuracy:.04f}.h5', 
                                                save_weights_only=True, period=5, monitor='accuracy')

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50, callbacks=[cp_callback])

Train on 25724 samples
Epoch 1/50
25724/25724 [==============================] - 3s 132us/sample - loss: 0.5338 - accuracy: 0.7399
Epoch 2/50
25724/25724 [==============================] - 3s 135us/sample - loss: 0.5342 - accuracy: 0.7407
Epoch 3/50
25724/25724 [==============================] - 4s 142us/sample - loss: 0.5339 - accuracy: 0.7401
Epoch 4/50
25724/25724 [==============================] - 4s 145us/sample - loss: 0.5338 - accuracy: 0.7403
Epoch 5/50
25724/25724 [==============================] - 4s 158us/sample - loss: 0.5334 - accuracy: 0.7402
Epoch 6/50
25724/25724 [==============================] - 4s 163us/sample - loss: 0.5335 - accuracy: 0.7401
Epoch 7/50
25724/25724 [==============================] - 5s 184us/sample - loss: 0.5339 - accuracy: 0.7395
Epoch 8/50
25724/25724 [==============================] - 4s 141us/sample - loss: 0.5336 - accuracy: 0.7401
Epoch 9/50
25724/25724 [==============================] - 4s 153us/sample - loss: 0.5337 - accuracy: 0.7402
Epoch

In [145]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 1s - loss: 0.5291 - accuracy: 0.7283
Loss: 0.5770398330236662, Accuracy: 0.7282798886299133


In [149]:
# Export our model to HDF5 file
!mkdir -p training_1_saved_model
nn.save('training_1_saved_model/AlphabetSoupCharity')

INFO:tensorflow:Assets written to: training_1_saved_model/AlphabetSoupCharity/assets


In [150]:
# check to see if model is saved correctly
new_model = tf.keras.models.load_model('training_1_saved_model/model_1')

new_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 86)                3784      
_________________________________________________________________
dense_13 (Dense)             (None, 43)                3741      
_________________________________________________________________
dense_14 (Dense)             (None, 21)                924       
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 22        
Total params: 8,471
Trainable params: 8,471
Non-trainable params: 0
_________________________________________________________________
